In [2]:
import time
import pandas as pd
from tqdm.notebook import tqdm
from selenium import webdriver
import numpy as np 
from pathlib import Path
from unidecode import unidecode
import requests, json

path = 'data'

root_folder = Path(path)
root_folder.mkdir(parents=True, exist_ok=True)

In [3]:
df_searchterms = pd.read_csv(root_folder /'medium_search_base.csv', sep = ';') 
list_dfs = []
for record in df_searchterms.to_dict(orient = 'records'):
    full_path_origin_folder = root_folder / 'raw_data'/ 'list_items' / record['folder']
    lst_jsons = list((full_path_origin_folder ).rglob("*.json")) 
    items = []
    for json_file_path in lst_jsons:
        if int(json_file_path.name.split('.json')[0]) > 20:
            continue
        with open(json_file_path, mode = 'rb') as file:
            r =file.read()
            json_file  = json.loads(r)
        items += json_file[0]['data']['search']['posts']['items']

    df = pd.DataFrame(items)
    for k,v in record.items():
        df[k] = v
    list_dfs.append(df)

df_searchterms = pd.concat(list_dfs, axis = 0, ignore_index=True)
df_searchterms.sort_values(by = ['folder','voterCount','mediumUrl'],\
                            ascending=[False,False,False], ignore_index=True, inplace=True)
df_searchterms.drop_duplicates(subset='algoliaObjectId',keep='first', inplace = True)
df_searchterms['path_folder'] = path + '\\articles\\'+ df_searchterms['folder']
df_searchterms['path_file'] = df_searchterms['path_folder']+ '\\'+df_searchterms['algoliaObjectId']+'.html'

In [7]:
df_searchterms.to_csv(root_folder /'medium_search_base_detail.csv', sep = ';')

In [4]:
driver = webdriver.Chrome()
url = df_searchterms['mediumUrl'].values[0]
driver.get(url)

In [ ]:
records = df_searchterms.to_dict(orient = 'records')
for record in tqdm(records):

    full_path_folder = Path(record['path_folder'])
    full_path_folder.mkdir(parents=True, exist_ok=True)

    full_path = Path(record['path_file'])

    if full_path.exists() and full_path.is_file():  
        continue
    
    url = record['mediumUrl']
    if not isinstance(url, str):
        continue 

    try:
        driver.get(url)
        time.sleep(4)
        alpha, beta = 2,5
        val = np.random.beta(alpha, beta)
        wait_time = 1 + val * (3 - 1)
        time.sleep(wait_time) 

        if 'Confirme que você é humano' in driver.page_source:
            break

        if 'Verificando se você é humano' in driver.page_source:
            break

        if 'There was a problem providing the content you requested' in driver.page_source:
            break

        with open(record['path_file'], "w", encoding="utf-8") as f:
            f.write(driver.page_source)
            
    except Exception as e:
            print('erro em ',url)
            print(e)
            error = True

    

  0%|          | 0/2626 [00:00<?, ?it/s]